In [1]:
print("This is the Code which is used to import the historical data used for the Poisson Distribution. NOTE: ALL JDBC CONNECTIONS / IMPORTS HAVE BEEN COMMENTED OUT.")

This is the Code which is used to import the historical data used for the Poisson Distribution. NOTE: ALL JDBC CONNECTIONS / IMPORTS HAVE BEEN COMMENTED OUT.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam
# importing the tools required for the Poisson regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf
import urllib.request
from bs4 import BeautifulSoup

epl_full = pd.read_csv("http://www.football-data.co.uk/mmz4281/1920/E0.csv")
epl = epl_full[['HomeTeam','AwayTeam','FTHG','FTAG']]
epl = epl.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})

Historical = spark.createDataFrame(epl_full)
##jdbcUrl = "INSERT JDBC CONNECTION" 
##Historical.write.mode('overwrite').jdbc(url=jdbcUrl, table="TEMP TABLE HERE")

epl.mean()

Out[ 1 ]: HomeGoals 1.572327
AwayGoals 1.352201
dtype: float64

In [3]:
# construct Poisson  for each mean goals value
poisson_pred = np.column_stack([[poisson.pmf(i, epl.mean()[j]) for i in range(8)] for j in range(2)])

# plot histogram of actual goals
plt.hist(epl[['HomeGoals', 'AwayGoals']].values, range(9), 
         alpha=0.7, label=['Home', 'Away'],normed=True, color=["#FFA07A", "#20B2AA"])

# add lines for the Poisson distributions
pois1, = plt.plot([i-0.5 for i in range(1,9)], poisson_pred[:,0],
                  linestyle='-', marker='o',label="Home", color = '#CD5C5C')
pois2, = plt.plot([i-0.5 for i in range(1,9)], poisson_pred[:,1],
                  linestyle='-', marker='o',label="Away", color = '#006400')

leg=plt.legend(loc='upper right', fontsize=13, ncol=2)
leg.set_title("Poisson           Actual        ", prop = {'size':'14', 'weight':'bold'})

plt.xticks([i-0.5 for i in range(1,9)],[i for i in range(9)])
plt.xlabel("Goals per Match",size=13)
plt.ylabel("Proportion of Matches",size=13)
plt.title("Number of Goals per Match (EPL 2019/20 Season)",size=14,fontweight='bold')
plt.ylim([-0.004, 0.4])
plt.tight_layout()
plt.show()

In [4]:
# probability of draw between home and away team
draw_odds = skellam.pmf(0.0,  epl.mean()[0],  epl.mean()[1])
print("probability of draw between home and away team for the entire league", draw_odds)


# probability of home team winning by one goal
win_by_1 = skellam.pmf(1,  epl.mean()[0],  epl.mean()[1])
print("probability of home team winning by one goal", win_by_1)


probability of draw between home and away team for the entire league 0.244924195515
probability of home team winning by one goal 0.212236677256

In [5]:
skellam_pred = [skellam.pmf(i,  epl.mean()[0],  epl.mean()[1]) for i in range(-6,8)]

plt.hist(epl[['HomeGoals']].values - epl[['AwayGoals']].values, range(-6,8), 
         alpha=0.7, label='Actual',normed=True)
plt.plot([i+0.5 for i in range(-6,8)], skellam_pred,
                  linestyle='-', marker='o',label="Skellam", color = '#CD5C5C')
plt.legend(loc='upper right', fontsize=13)
plt.xticks([i+0.5 for i in range(-6,8)],[i for i in range(-6,8)])
plt.xlabel("Home Goals - Away Goals",size=13)
plt.ylabel("Proportion of Matches",size=13)
plt.title("Difference in Goals Scored (Home Team vs Away Team)",size=14,fontweight='bold')
plt.ylim([-0.004, 0.26])
plt.tight_layout()
plt.show()

In [6]:
fig,(ax1,ax2) = plt.subplots(2, 1)


team1_home = epl[epl['HomeTeam']=='Chelsea'][['HomeGoals']].apply(pd.value_counts,normalize=True)
team1_home_pois = [poisson.pmf(i,np.sum(np.multiply(team1_home.values.T,team1_home.index.T),axis=1)[0]) for i in range(8)]
team2_home = epl[epl['HomeTeam']=='Liverpool'][['HomeGoals']].apply(pd.value_counts,normalize=True)
team2_home_pois = [poisson.pmf(i,np.sum(np.multiply(team2_home.values.T,team2_home.index.T),axis=1)[0]) for i in range(8)]

team1_away = epl[epl['AwayTeam']=='Chelsea'][['AwayGoals']].apply(pd.value_counts,normalize=True)
team1_away_pois = [poisson.pmf(i,np.sum(np.multiply(team1_away.values.T,team1_away.index.T),axis=1)[0]) for i in range(8)]
team2_away = epl[epl['AwayTeam']=='Liverpool'][['AwayGoals']].apply(pd.value_counts,normalize=True)
team2_away_pois = [poisson.pmf(i,np.sum(np.multiply(team2_away.values.T,team2_away.index.T),axis=1)[0]) for i in range(8)]

#ax1.bar(team1_home.index-0.4,team1_home.values,width=0.4,color="#034694",label="Chelsea")
#ax1.bar(team2_home.index,team2_home.values,width=0.4,color="#EB172B",label="Liverpool")
pois1, = ax1.plot([i for i in range(8)], team1_home_pois,
                  linestyle='-', marker='o',label="Home Team", color = "#0a7bff")
pois1, = ax1.plot([i for i in range(8)], team2_home_pois,
                  linestyle='-', marker='o',label="Away Team", color = "#ff7c89")
leg=ax1.legend(loc='upper right', fontsize=12, ncol=2)
leg.set_title("Poisson                 Actual                ", prop = {'size':'14', 'weight':'bold'})
ax1.set_xlim([-0.5,7.5])
ax1.set_ylim([-0.01,0.65])
ax1.set_xticklabels([])
# mimicing the facet plots in ggplot2 with a bit of a hack
ax1.text(7.65, 0.585, '                Home                ', rotation=-90,
        bbox={'facecolor':'#ffbcf6', 'alpha':0.5, 'pad':5})
ax2.text(7.65, 0.585, '                Away                ', rotation=-90,
        bbox={'facecolor':'#ffbcf6', 'alpha':0.5, 'pad':5})

#ax2.bar(team1_away.index-0.4,team1_away.values,width=0.4,color="#034694",label="Chelsea")
#ax2.bar(team2_away.index,team2_away.values,width=0.4,color="#EB172B",label="Sunderland")
pois1, = ax2.plot([i for i in range(8)], team1_away_pois,
                  linestyle='-', marker='o',label="Home Team", color = "#0a7bff")
pois1, = ax2.plot([i for i in range(8)], team2_away_pois,
                  linestyle='-', marker='o',label="Away Team", color = "#ff7c89")
ax2.set_xlim([-0.5,7.5])
ax2.set_ylim([-0.01,0.65])
ax1.set_title("Number of Goals per Match (EPL 2019/20 Season)",size=14,fontweight='bold')
ax2.set_xlabel("Goals per Match",size=13)
ax2.text(-1.15, 0.9, 'Proportion of Matches', rotation=90, size=13)
plt.tight_layout()
plt.show()

In [7]:
# importing the tools required for the Poisson regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf
n_columns = 0
n_rows=0
goal_model_data = pd.concat([epl[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
           epl[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
pd.DataFrame(columns = range(0,5), index= range(0,n_rows))

poisson_model.summary()

Generalized Linear Model Regression Results 
 
 Dep. Variable: goals No. Observations: 318 
 
 
 Model: GLM Df Residuals: 278 
 
 
 Model Family: Poisson Df Model: 39 
 
 
 Link Function: log Scale: 1.0 
 
 
 Method: IRLS Log-Likelihood: -438.57 
 
 
 Date: Mon, 09 Dec 2019 Deviance: 275.55 
 
 
 Time: 19:18:46 Pearson chi2: 226. 
 
 
 No. Iterations: 8 
 
 
 
 
 coef std err z P>|z| [95.0% Conf. Int.] 
 
 
 Intercept 0.3761 0.313 1.200 0.230 -0.238 0.990 
 
 
 team[T.Aston Villa] 0.0442 0.305 0.145 0.885 -0.554 0.642 
 
 
 team[T.Bournemouth] -0.2204 0.324 -0.680 0.496 -0.855 0.414 
 
 
 team[T.Brighton] -0.1723 0.315 -0.547 0.584 -0.790 0.445 
 
 
 team[T.Burnley] -0.0927 0.312 -0.298 0.766 -0.704 0.518 
 
 
 team[T.Chelsea] 0.3280 0.285 1.151 0.250 -0.230 0.886 
 
 
 team[T.Crystal Palace] -0.4831 0.348 -1.389 0.165 -1.165 0.199 
 
 
 team[T.Everton] -0.1897 0.318 -0.596 0.551 -0.813 0.434 
 
 
 team[T.Leicester] 0.5352 0.273 1.963 0.050 0.001 1.069 
 
 
 team[T.Liverpool] 0.5580 0.272 2.050 0.040 0.025 1.091 
 
 
 team[T.Man City] 0.6039 0.267 2.263 0.024 0.081 1.127 
 
 
 team[T.Man United] 0.1265 0.299 0.423 0.672 -0.459 0.712 
 
 
 team[T.Newcastle] -0.2960 0.328 -0.901 0.367 -0.940 0.348 
 
 
 team[T.Norwich] -0.3042 0.329 -0.924 0.355 -0.949 0.341 
 
 
 team[T.Sheffield United] -0.1948 0.319 -0.611 0.541 -0.820 0.430 
 
 
 team[T.Southampton] -0.2074 0.324 -0.641 0.522 -0.842 0.427 
 
 
 team[T.Tottenham] 0.2940 0.287 1.024 0.306 -0.269 0.857 
 
 
 team[T.Watford] -0.9426 0.401 -2.353 0.019 -1.728 -0.157 
 
 
 team[T.West Ham] -0.2337 0.328 -0.712 0.476 -0.877 0.410 
 
 
 team[T.Wolves] -0.0020 0.305 -0.007 0.995 -0.600 0.595 
 
 
 opponent[T.Aston Villa] 0.0206 0.284 0.072 0.942 -0.537 0.578 
 
 
 opponent[T.Bournemouth] -0.0739 0.294 -0.252 0.801 -0.650 0.502 
 
 
 opponent[T.Brighton] -0.1348 0.295 -0.457 0.647 -0.712 0.443 
 
 
 opponent[T.Burnley] 0.0915 0.282 0.324 0.746 -0.462 0.645 
 
 
 opponent[T.Chelsea] -0.0400 0.294 -0.136 0.892 -0.616 0.536 
 
 
 opponent[T.Crystal Palace] -0.4137 0.317 -1.304 0.192 -1.035 0.208 
 
 
 opponent[T.Everton] 0.0597 0.283 0.211 0.833 -0.496 0.615 
 
 
 opponent[T.Leicester] -0.8911 0.380 -2.342 0.019 -1.637 -0.145 
 
 
 opponent[T.Liverpool] -0.6128 0.341 -1.796 0.073 -1.282 0.056 
 
 
 opponent[T.Man City] -0.2198 0.312 -0.704 0.481 -0.831 0.392 
 
 
 opponent[T.Man United] -0.3442 0.312 -1.102 0.270 -0.956 0.268 
 
 
 opponent[T.Newcastle] -0.1790 0.297 -0.603 0.547 -0.761 0.403 
 
 
 opponent[T.Norwich] 0.3258 0.274 1.190 0.234 -0.211 0.863 
 
 
 opponent[T.Sheffield United] -0.4418 0.328 -1.348 0.178 -1.084 0.201 
 
 
 opponent[T.Southampton] 0.2589 0.271 0.955 0.339 -0.272 0.790 
 
 
 opponent[T.Tottenham] -0.1066 0.298 -0.358 0.720 -0.690 0.477 
 
 
 opponent[T.Watford] 0.1585 0.281 0.564 0.572 -0.392 0.709 
 
 
 opponent[T.West Ham] 0.0984 0.292 0.337 0.736 -0.474 0.671 
 
 
 opponent[T.Wolves] -0.2632 0.314 -0.839 0.401 -0.878 0.351 
 
 
 home 0.1450 0.094 1.548 0.122 -0.039 0.328

In [8]:
import urllib.request
from bs4 import BeautifulSoup
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

url = "https://www.oddschecker.com/football/english/premier-league"
headers={'User-Agent':user_agent,} 

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read() # The data u need
soup = BeautifulSoup(data , "lxml")
table = soup.find_all('table')[0]


    # Find number of rows and columns
# we also find the column titles if we can
for row in table.find_all('tr'):

    # Determine the number of rows in the table
    td_tags = row.find_all('p')
    if len(td_tags) > 0:
        n_rows+=1
        if n_columns == 0:
            # Set the number of columns for our table
            n_columns = len(td_tags)

df = pd.DataFrame(columns = range(0,6),
                  index= range(0,n_rows))
row_marker = 0
identity = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('p')
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
        
    if len(columns) > 0:
        df.iat[row_marker,column_marker] = identity
        identity += 1
        row_marker += 1
        
header = ['hometeam','awayteam','homeodds','drawodds','awayodds', 'id']
df.columns=(header)


In [9]:
hometeam = []
awayteam = []
homeodds = []
drawodds = []
awayodds = []
id = []
all_names = list(set(epl["HomeTeam"]))

for (hometeam_v, awayteam_v, home_fraction, draw_fraction, away_fraction, ids) in zip(df['hometeam'],df['awayteam'],df['homeodds'],df['drawodds'],df['awayodds'], df['id']):
  for (cleanname,name) in zip(all_names,df['hometeam']):
    if hometeam_v[:5] == cleanname[:5]:
      hometeam.append(cleanname)
    if awayteam_v[:5] == cleanname[:5]:
      awayteam.append(cleanname)
  homeodd_raw = eval(home_fraction)
  homeodds_v = (1/(homeodd_raw+ 1))
  homeodds.append(homeodds_v)
  drawodd_raw = eval(draw_fraction)
  drawodd_raw_v = (1/(drawodd_raw+ 1))
  drawodds.append(drawodd_raw_v)
  awayodds_raw = eval(away_fraction)
  awayodds_v = (1/(awayodds_raw+ 1))
  awayodds.append(awayodds_v)
  id.append(ids)

  
percentagedf = pd.DataFrame(list(zip(hometeam, awayteam, homeodds, drawodds,awayodds,id)),
  columns=['hometeam','awayteam','homeodds','drawodds','awayodds', 'id'])

In [10]:
percentagedf

,hometeam,awayteam,homeodds,drawodds,awayodds,id
0,West Ham,Arsenal,0.274194,0.230769,0.491525,0
1,Liverpool,Watford,0.818182,0.129870,0.058824,1
2,Burnley,Newcastle,0.476190,0.277778,0.255319,2
3,Chelsea,Bournemouth,0.750000,0.152778,0.083333,3
4,Leicester,Norwich,0.800000,0.140845,0.066667,4
5,Sheffield United,Aston Villa,0.500000,0.260274,0.232558,5
6,Southampton,West Ham,0.512821,0.246753,0.256410,6
7,Man United,Everton,0.551724,0.263158,0.197917,7
8,Wolves,Tottenham,0.294118,0.275000,0.434783,8
9,Arsenal,Man City,0.160000,0.181818,0.666667,9


In [11]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam,'opponent': awayTeam,'home':1},index=[1]))[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1]))[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

hometeam = []
awayteam = []
home_prob = []
draw_prob = []
away_prob = []
idprob = []

ids = 0
for (ht,at) in zip(percentagedf.iloc[:, 0], percentagedf.iloc[:, 1]):
    if ht.find("Utd") >=0:
        ht= ht.replace("Utd", "United")
        at = at.replace("Utd", "United")
    elif at.find("Utd") >=0:
        ht= ht.replace("Utd", "United")
        at = at.replace("Utd", "United")
    odds_matrix = simulate_match(poisson_model, ht, at, max_goals = 10)
    home_probability = np.sum(np.tril(odds_matrix, -1))
    draw_probability = np.sum(np.diag(odds_matrix))
    away_probability = np.sum(np.triu(odds_matrix, 1))
    hometeam.append(ht)
    awayteam.append(at)
    home_prob.append(home_probability)
    draw_prob.append(draw_probability)
    away_prob.append(away_probability)
    idprob.append(ids)
    ids +=1

percentile_list = pd.DataFrame(
    {'id': idprob,
     'hometeam': hometeam,
     'awayteam': awayteam,
     'home_prob': home_prob,
     'draw_prob': draw_prob,
     'away_prob': away_prob
    })

Total_Odds = percentile_list.merge(percentagedf, on = 'id')

In [12]:
Total_Odds

,away_prob,awayteam_x,draw_prob,home_prob,hometeam_x,id,hometeam_y,awayteam_y,homeodds,drawodds,awayodds
0,0.439058,Arsenal,0.243287,0.317654,West Ham,0,West Ham,Arsenal,0.274194,0.230769,0.491525
1,0.013459,Watford,0.055616,0.930021,Liverpool,1,Liverpool,Watford,0.818182,0.129870,0.058824
2,0.341233,Newcastle,0.271564,0.387202,Burnley,2,Burnley,Newcastle,0.476190,0.277778,0.255319
3,0.187560,Bournemouth,0.200152,0.612271,Chelsea,3,Chelsea,Bournemouth,0.750000,0.152778,0.083333
4,0.013801,Norwich,0.044286,0.939161,Leicester,4,Leicester,Norwich,0.800000,0.140845,0.066667
5,0.260634,Aston Villa,0.268880,0.470485,Sheffield United,5,Sheffield United,Aston Villa,0.500000,0.260274,0.232558
6,0.375033,West Ham,0.242824,0.382142,Southampton,6,Southampton,West Ham,0.512821,0.246753,0.256410
7,0.149049,Everton,0.204035,0.646906,Man United,7,Man United,Everton,0.551724,0.263158,0.197917
8,0.376974,Tottenham,0.242406,0.380618,Wolves,8,Wolves,Tottenham,0.294118,0.275000,0.434783
9,0.653825,Man City,0.170861,0.175206,Arsenal,9,Arsenal,Man City,0.160000,0.181818,0.666667


In [13]:
dfOdds = spark.createDataFrame(Total_Odds)

##jdbcUrl = "INSERT JDBC HERE"

##dfOdds.write.mode('overwrite').jdbc(url=jdbcUrl, table="TABLE NAME HERE")